# Probability distribution derivator_v5

#### 파라미터 변경
#### 20221019. 분포형태 추가(파라미터 값 계산 가능한 것으로)
#### 20230607. 확률분포 유형 변경 및 파라미터 계산 반영 +포장공

In [20]:
import numpy as np
import pandas as pd
import scipy
from scipy import stats

In [21]:
delay_pd = pd.read_csv('dataset_v5/음성충주 1,3,4,5공구 포장공 리스크 기초자료_DEC_수정_230602.csv')
delay = []
pre_data = []
pre_temp = []
cal_data = []
cal_temp = []

In [22]:
l_L3 = ["토공","포장공"]
l_L4 = ["본선","IC/JC","지선/부체도로"]
l_L7_1 = ["흙깎기","흙쌓기","토공기타","아스팔트","콘크리트","포장기타"]
l_L7_2 = [["토사","리핑","발파","흙깎기기타"],["노체","노상","흙쌓기기타"],["비탈면보호공","연약지반처리","보강토옹벽","콘크리트옹벽","옹벽기타","토공기타"],["동상방지층","린콘크리트포장","콘크리트포장","콘크리트포장기타"],["동상방지층","보조기층","기층","중간층","표층","아스팔트포장기타"],["특수포장","포장기타"]]
l_Risk = ["소음/진동","용지","진입로","기타","물가변동","지급신청","수급변경","업체변경","문제검토","문제협의","설계변경","시공계획","업무","검토","문화재","일반시설","유해/분묘","건설폐기물","임목폐기물","폐수","우천","태풍","폭염","폭설","인적","물적","기타"]

In [23]:
for i in l_L4:
    for j in l_L7_1:
        for k in l_L7_2[l_L7_1.index(j)]:
            for l in l_Risk:
                search_L4 = delay_pd['L4'] == i
                s_L4 = delay_pd[search_L4]
                search_L7_1 = s_L4['L7-1'] == j
                s_L7_1 = s_L4[search_L7_1]
                search_L7_2 = s_L7_1['L7-2'] == k
                s_L7_2 = s_L7_1[search_L7_2]
                search_Risk = s_L7_2['중'] == l
                s_Risk = s_L7_2[search_Risk]
                
                pre_temp = []
                if s_Risk['중'].empty==False:
                    pre_temp.append(s_Risk['No'])
                    pre_temp.append(s_Risk['L3'])       
                    pre_temp.append(s_Risk['L4'])
                    pre_temp.append(s_Risk['L7-1'])
                    pre_temp.append(s_Risk['L7-2'])
                    pre_temp.append(s_Risk['대'])
                    pre_temp.append(s_Risk['중'])
                    pre_temp.append(s_Risk['실제공기'])
                    pre_temp.append(s_Risk['공기지연'])
                    pre_temp.append(s_Risk['공기지연'].count())
                    pre_temp.append(s_Risk['공기지연'].mean())
                    pre_temp.append(s_Risk['공기지연'].var())
                    pre_temp.append(s_Risk['공기지연'].std())
                    pre_temp.append(s_Risk['공기지연'].max())
                    pre_temp.append(s_Risk['공기지연'].min())
                    pre_data.append(pre_temp)

                cal_temp = []
                if s_Risk['중'].empty==False:
                    cal_temp.append(s_Risk['No'].values)
                    cal_temp.append(s_Risk['L3'].values)       
                    cal_temp.append(s_Risk['L4'].values)
                    cal_temp.append(s_Risk['L7-1'].values)
                    cal_temp.append(s_Risk['L7-2'].values)
                    cal_temp.append(s_Risk['대'].values)
                    cal_temp.append(s_Risk['중'].values)
                    cal_temp.append(s_Risk['실제공기'].values)
                    cal_temp.append(s_Risk['공기지연'].values)
                    cal_temp.append(s_Risk['공기지연'].count())
                    cal_temp.append(s_Risk['공기지연'].mean())
                    cal_temp.append(s_Risk['공기지연'].var())
                    cal_temp.append(s_Risk['공기지연'].std())
                    cal_temp.append(s_Risk['공기지연'].max())
                    cal_temp.append(s_Risk['공기지연'].min())
                    cal_data.append(cal_temp)

In [24]:
# df=pd.DataFrame(pre_data)
# df.to_csv("dataset_v5/preset_data_v5.csv", encoding='utf-8-sig')
# df=pd.DataFrame(cal_data)
# df.to_csv("dataset_v5/cal_data_v5.csv", encoding='utf-8-sig')

In [25]:
new_temp = []
new_data = []
parm_temp = []
parm_data = []
#new_data = [['공종\n(L3)','시설물\n(L4)','작업단위1\n(L7)','작업단위2\n(L7)','리스크종류','공기','리스크발생확률\n(%)','확률분포','P-value','리스크횟수','평균\n(지연일수)','분산\n(지연일수)','표준편차\n(지연일수)','최댓값\n(지연일수)','최솟값\n(지연일수)']]
dist_names = ["norm", "gamma", "expon", "t", "chi2"]
dist_results = []
params = {}
count = 0

In [26]:
for i in range(0, len(cal_data)):
    count = 0
    new_temp = []
    count = count + cal_data[i][7][0]
    for j in range(1, len(cal_data[i][7])):
        if cal_data[i][7][j-1]!=cal_data[i][7][j]:
            count = count + cal_data[i][7][j]
    for k in range(0, len(cal_data[i])):
        if k == 7:
            new_temp.append(count) 
        elif k == 8:
            data = cal_data[i][8]
            new_temp.append(cal_data[i][9]*100/count)
            dist_results = []
            params = {}
            for dist_name in dist_names:
                dist = getattr(scipy.stats, dist_name)
                param = dist.fit(data)
                params[dist_name] = param
                D, p = stats.kstest(data, dist_name, param)
                dist_results.append((dist_name, p))
            best_dist, best_p = (max(dist_results, key=lambda item: item[1])) 
            new_temp.append(best_dist)
            new_temp.append(best_p)
        elif k >= 9:
            new_temp.append(cal_data[i][k])
        else:
            if k != 0:
                new_temp.append(cal_data[i][k][0])
    new_data.append(new_temp)


/home/min/.local/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1844: RuntimeWarning: invalid value encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/home/min/.local/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:400: RuntimeWarning: invalid value encountered in double_scalars
  return m3 / np.power(m2, 1.5)


In [12]:
# df=pd.DataFrame(new_data)
# df.columns=[['공종\n(L3)','시설물\n(L4)','작업단위1\n(L7)','작업단위2\n(L7)','리스크종류(대)','리스크종류(중)','공기','리스크발생확률\n(%)','확률분포','P-value','리스크횟수','평균\n(지연일수)','분산\n(지연일수)','표준편차\n(지연일수)','최댓값\n(지연일수)','최솟값\n(지연일수)']]
# df=df[['공종\n(L3)','시설물\n(L4)','작업단위1\n(L7)','작업단위2\n(L7)','리스크종류(대)','리스크종류(중)','공기','리스크횟수','리스크발생확률\n(%)','확률분포','P-value','평균\n(지연일수)','분산\n(지연일수)','표준편차\n(지연일수)','최댓값\n(지연일수)','최솟값\n(지연일수)']]
# df.to_csv("dataset_v5/리스크 확률분포_파라미터입력전_20230607.csv", encoding='utf-8-sig')

In [27]:
len(new_data)

15

In [45]:
print(new_data[0][8]) #확률분포
print(new_data[0][9]) #p-value
print(new_data[0][10]) #리스크횟수
print(new_data[0][11]) #평균
print(new_data[0][12]) #분산
print(new_data[0][13]) #표준편차
print(new_data[0][14]) #최댓값
print(new_data[0][15]) #최솟값

t
0.7161089508423335
4
1.75
0.9166666666666666
0.9574271077563381
3
1


In [46]:
# "norm", "gamma", "expon", "t", "chi2"
for i in range(0, len(new_data)):
    parm_temp = new_data[i]
    if new_data[i][8] == 'norm':
        parm1 = new_data[i][11] #E(x)
        parm2 = new_data[i][13] #sigma(x)
        parm_temp.append(parm1)
        parm_temp.append(parm2)

    elif new_data[i][8] == 'gamma':
        e = new_data[i][11] #평균
        s = new_data[i][13] #표준편차
        parm1 = e*e/s*s #shape
        parm2 = e/s*s #inverse scale
        parm_temp.append(parm1)
        parm_temp.append(parm2)

    elif new_data[i][8] == 'expon':
        e = new_data[i][11] #평균
        parm2 = 1/e
        parm_temp.append('')
        parm_temp.append(parm2)

    elif new_data[i][8] == 't':
        num = new_data[i][10] #표본갯수
        parm1 = num-1
        parm_temp.append(parm1)
        parm_temp.append('')

    elif new_data[i][8] == 'chi2':
        num = new_data[i][10] #표본갯수
        parm1 = num-1
        parm_temp.append(parm1)
        parm_temp.append('')

    parm_data.append(parm_temp)


In [48]:
df=pd.DataFrame(parm_data)
df.columns=[['공종\n(L3)','시설물\n(L4)','작업단위1\n(L7)','작업단위2\n(L7)','리스크종류(대)','리스크종류(중)','공기','리스크발생확률\n(%)','확률분포','P-value','리스크횟수','평균\n(지연일수)','분산\n(지연일수)','표준편차\n(지연일수)','최댓값\n(지연일수)','최솟값\n(지연일수)','param1','param2']]
df=df[['공종\n(L3)','시설물\n(L4)','작업단위1\n(L7)','작업단위2\n(L7)','리스크종류(대)','리스크종류(중)','공기','리스크횟수','리스크발생확률\n(%)','확률분포','P-value','param1','param2','평균\n(지연일수)','분산\n(지연일수)','표준편차\n(지연일수)','최댓값\n(지연일수)','최솟값\n(지연일수)']]
df.to_csv("dataset_v5/음성충주 1,3,4,5 공구_포장공_리스크 확률분포_20230607.csv", encoding='utf-8-sig')